In [50]:
import cvxpy as cp 

 #Luca Silva Celiberto

# Constants 
weeks = range(1, 53)  # Weeks 1 to 52 
products = ["apples", "bananas", "oranges"] # Products

 

mu = {"apples": 1.85, "oranges": 1.46, "bananas": 0.60}  # Prices (USD)

costs = {"apples": 0.56, "oranges": 0.29, "bananas": 0.15}  # Costs (USD)

space_per_pound = {"apples": 0.04, "oranges": 0.03, "bananas": 0.05}  # Space per lb. of product (cubic ft)

base_demand = { # Base demand for product (lbs/wk)
    
    "apples": [2596] * 34 + [2856] * 11 + [2596] * 7, 
    
    "oranges": [865] * 5 + [952] * 41 + [865] * 6, 

    "bananas": [3750] * 52, 

} 

alpha = {"apples": 0.05, "oranges": 0.05, "bananas": 0.05} # Minimum percentage of display space for each product


s = 450  # Total space in cubic feet 

w = {"apples": 0.25, "oranges": 0.25, "bananas": 0.25} # Maximum percentage of weekly shipment for each product stored
 

# Elasticity functions 

def q(product, price): 
    
    adj_price = 0

    if product == "apples": 

        adj_price = 1 - 0.58 * ((price - mu["apples"]) / mu["apples"]) 

    elif product == "oranges": 

        adj_price = 1 - 1.10 * ((price - mu["oranges"]) / mu["oranges"]) 

    elif product == "bananas": 

        adj_price =  1 - 1.01 * ((price - mu["bananas"]) / mu["bananas"]) 
    
    # Demand cannot be negative
    if adj_price < 0: adj_price = 0
    
    return adj_price

 

# Decision variables 

x = { (i, j): cp.Variable(nonneg=True) for i in weeks for j in products } 

y = { (i, j): cp.Variable(nonneg=True) for i in weeks for j in products } 

 

# Objective function 

profit = cp.sum([x[i, j] * (mu[j] - costs[j]) for i in weeks for j in products]) 

objective = cp.Maximize(profit)
 

# Constraints 

constraints = [] 

 



for i in weeks: 

    for j in products: 

        # Demand satisfaction constraint (1)
        price_adjusted_demand = q(j, mu[j]) * base_demand[j][i - 1]
        
        prev_l = y[(i - 1, j)] if i > 1 else 0 
        
        constraints.append( 

            x[(i, j)] + prev_l - y[(i, j)] == price_adjusted_demand

        ) 
        
        # Minimum display percentage constraint (3)
        constraints.append((x[(i, j)] * space_per_pound[j]) / s >= alpha[j]) 
        
        # Maximum storage percentage constraint (4)
        constraints.append(y[(i, j)] <= w[j] * x[(i, j)])



# Space constraint (2)

for i in weeks: 

    constraints.append( 
        
        cp.sum([(x[(i, j)] + (y[(i - 1, j)] if i > 1 else 0) - y[(i, j)]) * space_per_pound[j] for j in products]) <= s

    ) 
    

# Boundary constraint for leftovers (5)

for j in products: 

    constraints.append(y[(52, j)] == 0) 

 

# Problem definition and solve 

problem = cp.Problem(objective, constraints) 

problem.solve(verbose=True, solver=cp.GUROBI) 



# Output 

if problem.status not in ["optimal", "optimal_inaccurate"]: 

    print(f"Problem status: {problem.status}") 

else: 

    for j in products: 

        print(f"Product: {j}") 

        for i in weeks: 

            print(f"Week {i}: Ordered = {x[(i, j)].value}, Stored = {y[(i, j)].value}")                         
     
            
    
# Audrey Beckman       

# Post optimality analysis: dual values
dual_1 = constraints[0].dual_value # constraint 1, week 1, apples
dual_2 = constraints[468].dual_value #constraint 2, week 1, apples
dual_3 = constraints[1].dual_value #constraint 3, week 1, apples
dual_4 = constraints[2].dual_value #constraint 4, week 1, apples


#Dual Output
print(f"Constraint 1 Dual Value: {dual_1}")
print(f"Constraint 2 Dual Value: {dual_2}")
print(f"Constraint 3 Dual Value: {dual_3}")
print(f"Constraint 4 Dual Value: {dual_4}")



# Mandy Qu

# Get baseline profit
baseline_profit = problem.solve(verbose=False, solver=cp.GUROBI)
print("\nBaseline Profit: ", baseline_profit)

# Price change by percentage
# Three scenarios
price_scenarios = {
    "Decrease by 10%": {j: mu[j] * 0.90 for j in products},
    "Increase by 10%": {j: mu[j] * 1.10 for j in products},
    "Increase by 50%": {j: mu[j] * 1.50 for j in products},
}

for scenario, prices in price_scenarios.items():
    # Calculate profit
    new_profit = cp.sum([x[i, j] * (prices[j] - costs[j]) for i in weeks for j in products])
    new_objective = cp.Maximize(new_profit)
    problem = cp.Problem(new_objective, constraints)
    # For each scenario, get profit
    scenario_profit = problem.solve(verbose=False, solver=cp.GUROBI)
    # Calculate difference between baseline and scenario profit
    profit_difference = scenario_profit - baseline_profit
    # Print scenario
    print(f"\nScenario: {scenario}")
    # Print scenario profit, and check if it's increase or decrease form baseline
    print(f"Profit Impact: {scenario_profit:.2f}")
    print(f"({'Increase' if profit_difference > 0 else 'Same' if profit_difference == 0 else 'Decrease'})")
    



# Difference in Space (s)
# For each scenario
for new_space in [400, 425, 450, 475, 490]:
    s = new_space
    problem = cp.Problem(objective, constraints)
    optimal_profit = problem.solve(verbose=False, solver=cp.GUROBI)
    profit_difference = optimal_profit - baseline_profit
    # print new space and the profit
    print(f"New Space: {new_space}, Optimal Profit: {optimal_profit}")
    print(f"({'Increase' if profit_difference > 0 else 'Same' if profit_difference == 0 else 'Decrease'})")

#resetting constant value
s = 450

# Demand increase by 20%
base_demand["bananas"] = [d * 1.20 for d in base_demand["bananas"]] 
problem = cp.Problem(objective, constraints)
scenario_profit = problem.solve(verbose=False, solver=cp.GUROBI)
# If unbounded, print this
if scenario_profit is None:
    print("Problem infeasible.")
else:
    # Calculate difference
    profit_difference = scenario_profit - baseline_profit
    # Print
    print(f"New Demand Scenario for Bananas, Optimal Profit: {scenario_profit:.2f}")
    print(f"({'Increase' if profit_difference > 0 else 'Same' if profit_difference == 0 else 'Decrease'})")
    


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 11:03:52 PM: Your problem has 312 variables, 523 constraints, and 0 parameters.
(CVXPY) Nov 25 11:03:52 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 11:03:52 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 11:03:52 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 11:03:52 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 11:03:52 PM: Compiling problem (target solver=GUROBI).